https://github.com/sharkwithkids/kakao.git

In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# CSV 파일에서 URL 리스트 가져오기
title_df = pd.read_csv("kakao_listly.csv")
id_list_total = title_df['url'].astype(str).tolist()

num_id_list_total = len(id_list_total)  # 2677
num_id_csv_files = num_id_list_total // 100  # 26

In [10]:
def make_data(id_list):

    # WebDriver 설정
    options = Options()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    driver = webdriver.Chrome(options=options)

    # 데이터 저장 리스트
    data = []

    # URL 순회하며 정보 수집
    for url in id_list:
        # 오버뷰 페이지 접속
        overview_url = f"{url}?tab_type=overview"
        driver.get(overview_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 제목
        title_span = soup.find("span", class_="font-large3-bold mb-3pxr text-ellipsis break-all text-el-70 line-clamp-2")
        title_text = title_span.get_text(strip=True) if title_span else "제목 없음"

        # 작가
        writer_span = soup.find("span", class_="font-small2 mb-6pxr text-ellipsis text-el-70 opacity-70 break-word-anywhere line-clamp-2")
        writer_text = writer_span.get_text(strip=True) if writer_span else "작가 없음"

        # 장르 (웹툰 제외)
        genre_spans = soup.find_all("span", class_="break-all align-middle")
        genres = [g.get_text(strip=True) for g in genre_spans]
        genre_text = genres[1] if len(genres) > 1 else "장르 없음"

        # 조회수 & 별점 (같은 클래스에서 0: 조회수, 1: 별점)
        view_rating_spans = soup.find_all("span", class_="text-el-70 opacity-70")
        view_count_text = view_rating_spans[0].get_text(strip=True) if len(view_rating_spans) > 0 else "조회수 없음"
        rating_text = view_rating_spans[1].get_text(strip=True) if len(view_rating_spans) > 1 else "별점 없음"

        # 회차수 & 댓글수 (같은 클래스에서 0: 회차수, 1: 댓글수)
        episode_comment_spans = soup.find_all("span", class_="text-ellipsis break-all line-clamp-1 font-small2-bold text-el-70")
        episode_count_text = episode_comment_spans[0].get_text(strip=True) if len(episode_comment_spans) > 0 else "회차수 없음"
        comment_count_text = episode_comment_spans[1].get_text(strip=True) if len(episode_comment_spans) > 1 else "댓글수 없음"

        # 어바웃 페이지 접속
        about_url = f"{url}?tab_type=about"
        driver.get(about_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 줄거리
        story_span = soup.find("span", class_="font-small1 mb-8pxr block whitespace-pre-wrap break-words text-el-70")
        story_text = story_span.get_text(strip=True) if story_span else "줄거리 없음"

        # 해시태그
        hashtags = []
        hashtag_spans = soup.find_all("span", class_="font-small2-bold text-ellipsis text-el-70 line-clamp-1")
        for tag in hashtag_spans:
            hashtags.append(tag.get_text(strip=True))
        hashtags_text = ', '.join(hashtags) if hashtags else "해시태그 없음"

        # 연령 등급
        age_rating_text = "연령 등급 없음"
        info_divs = soup.find_all("div", class_="font-small1 flex w-full pt-6pxr")
        for div in info_divs:
            label_span = div.find("span")
            if label_span and "연령등급" in label_span.get_text():
                value_span = div.find_all("span")[-1]
                age_rating_text = value_span.get_text(strip=True)
                break

        # 데이터 저장
        data.append({
            'Title': title_text,
            'Writer': writer_text,
            'Genre': genre_text,
            'View Count': view_count_text,
            'Rating': rating_text,
            'Episode Count': episode_count_text,
            'Comment Count': comment_count_text,
            'Story': story_text,
            'Hashtags': hashtags_text,
            'Age Rating': age_rating_text
        })

        # print(f"현재 작품: {title_text}")
        time.sleep(10)

    # 드라이버 종료
    driver.quit()

    return data

In [11]:
for idx in range(num_id_csv_files+1):  # idx = 0 .. 26+1
    start, end = idx*100, min(idx*100+100, num_id_list_total)
    id_list = id_list_total[start:end]
    data = make_data(id_list)
    
    # 데이터프레임 생성 및 CSV 저장
    df = pd.DataFrame(data)[[
        'Title', 'Writer', 'Genre', 'View Count', 'Rating',
        'Episode Count', 'Comment Count', 'Story', 'Hashtags', 'Age Rating'
    ]]
    df.to_csv(f'kakao_crawling_{idx}.csv', index=False, encoding='utf-8-sig')
    print(f'kakao_crawlling_{idx}.csv')

kakao_crawlling_0.csv
kakao_crawlling_1.csv
kakao_crawlling_2.csv
kakao_crawlling_3.csv
kakao_crawlling_4.csv
kakao_crawlling_5.csv
kakao_crawlling_6.csv
kakao_crawlling_7.csv
kakao_crawlling_8.csv
kakao_crawlling_9.csv
kakao_crawlling_10.csv
kakao_crawlling_11.csv
kakao_crawlling_12.csv
kakao_crawlling_13.csv
kakao_crawlling_14.csv
kakao_crawlling_15.csv
kakao_crawlling_16.csv
kakao_crawlling_17.csv
kakao_crawlling_18.csv
kakao_crawlling_19.csv
kakao_crawlling_20.csv
kakao_crawlling_21.csv
kakao_crawlling_22.csv
kakao_crawlling_23.csv
kakao_crawlling_24.csv
kakao_crawlling_25.csv
kakao_crawlling_26.csv


In [12]:
import pandas as pd

num_id_list_total = 2677
num_id_csv_files = num_id_list_total // 100  # 26
df_all = pd.DataFrame()

for idx in range(num_id_csv_files+1):  # idx = 0 .. 26+1
    filename = f'kakao_crawling_{idx}.csv'
    df = pd.read_csv(filename)
    df_all = pd.concat([df_all, df], axis=0)
    
df_all.to_csv('kakao_crawling_all.csv', index=False, encoding='utf-8-sig')
df_all.head()

,Title,Writer,Genre,View Count,Rating,Episode Count,Comment Count,Story,Hashtags,Age Rating
0,나 혼자만 레벨업,"현군,장성락(REDICE STUDIO),추공",판타지,5.5억,10.0,전체203,전체 41.6만,"10여 년 전, \n다른 차원과 이쪽 세계를 이어 주는 통로 ‘게이트’가 열리고 \...","#생존, #먼치킨, #시스템, #천재, #노력, #성장, #이능력, #플레이어",전체이용가
1,접근 불가 레이디,"용두식,시조새,ZI.O,밍숭,킨",로판,"4,608.8만",9.9,전체126,전체 3.1만,"""힐리스. 부탁이야. 네가 가브리엘 대신 죽어줘.""\n\n\n언제나 고귀하던 내 오...","#로맨스판타지, #복수물, #회귀물, #피폐물, #여주판타지, #운명, #까칠남, ...",전체이용가
2,제목 없음,작가 없음,장르 없음,조회수 없음,별점 없음,회차수 없음,댓글수 없음,줄거리 없음,해시태그 없음,연령 등급 없음
3,무협지 악녀인데 내가 제일 쎄!,"가비남,유아니,윌브라이트",로판,"3,354.1만",10.0,전체131,전체 2.9만,"""하필이면 주인공도 아니고,\n소설 속 제일가는 악녀라니…""\n\n월세 탈출을 위해...","#로맨스판타지, #빙의물, #무협물, #순정남, #능력남, #존댓말남, #대형견남,...",전체이용가
4,그녀와 야수,"홍슬,마지노선",로판,"4,363만",10.0,전체178,전체 3.5만,"제국의 시조였던 전설적인 인물 마티나의 죽음으로부터 백 년 후,\n마티나는 귀족가의...",#로맨스판타지,15세이용가
